In [55]:
# McKee_Scattering_2013
# Jesse Bausell
# September 10, 2020
#
# McKee_Scattering_2013 corrects ac-s data for undetected scattering inside the absorption and attenuation tubes based 
# on methods devised by McKee et al. (2013) (full citaiton below). These methods assumd different reflectance efficiencies
# (rw) of ac-s absorption tube. Therefore, the script outputs 10 different correction files, one for each of 10 reflectance 
# efficiencies ranging from 1 (100%) reflectance to 0.95 (95% reflectance). 
# IMPORTANT: this script cannot run without: MK13_coefFILE.h5
#
# Full citation: 
# McKee, D., and J. Piskozub et al. (2013) Evaluation and Improvement of an Iterative Scattering Correction Scheme for 
# in situ absorption and attenuation measurements. Journal of atmospheric and ocean technology 30:1527-1541

In [56]:
# Import libraries
import numpy as np
import h5py
from tkinter import filedialog as fd
from matplotlib import pyplot as plt
from datetime import datetime as dt 
import os
from distutils.dir_util import copy_tree

In [57]:
# 1. Define function for retrieving reflective tube specific coefficients.

def McKEE2013_COEFF(**kwargs):
    """This program is used to get the proper flow tube (r_w) specific McKee et al. (2013)
    scattering coefficients
    Keyword arguments (input):
        r_w - reflection efficiency of ac-s absorption tube. This variable is qualitative and must be one 
        of the following values: 1, 0.999, 0.998, 0.995, 0.99, 0.985, 0.98, 0.97, 0.96 , & 0.95  
    Outputs:
        coeff_array - tuple of absorption coefficients according to user-selected r_w. Based on McKee et al. (2013)"""
    ### 1. Create preliminary variables for acquiring correct scattering coefficients based on r_w 
    r_w = kwargs['r_w'] # Percent of scattering as input by user (based on keyword argument)
    coeff_DICT = {} # Empty dictionary
    pwd = os.getcwd()+'/' + 'MK13_coefFILE.h5'# Find present working directory and select MK13_coefFILE.h5
    ### 2. Open MK13_coefFILE.h5 and select correct set of coefficients
    with h5py.File(pwd) as hf:
        # Open the hdf5 file created in the previous cell
        for m in hf.keys():
            # for-loop fills in the dictionary
            coeff_DICT[m] = hf[m][()] # Place coefficients into empty dictionary
    ### 3. Create an array of coefficients for the proper 
    # Creates a matrix of nans in the dimensions of the a coefficients and the number of 
    # reflection efficiencies (r_w's) (below)
    COEFF_MAT = np.ones([len(coeff_DICT['a_0']),len(coeff_DICT.keys())-1])*np.nan 
    for i in np.arange(7):
        # This for-loop fills in the matrix with the correct coefficient values (a_0-a_6)
        COEFF_MAT[:,i] = coeff_DICT['a_' + str(i)] # Fill matrix by columns
    booL_IND = coeff_DICT['r_w'] == r_w # bool index the correct r_w as input by the user
    coeff_array = tuple(COEFF_MAT[booL_IND,:].reshape(7,)) # make a tuple of the correct matrix row
    return(coeff_array) # Output a tuple

In [58]:
# 2. Define functions needed for pre-processing ac-s data in preparation for McKee et al.'s 
# (2013) scattering correction approach 

def acs_matlab_LOAD(acs_fileNAME):
    """acs_matlab_LOAD loads .mat/hdf5 files of ac-s data that is partially processed by matlab.
    It relies on the program 'acsPROCESS_INTERACTIVE.m.
    Input variables:
        acs_fileNAME - ac-s data. Must be hdf5 file (also formatted like .mat file)
    Output variables:
        A_CORR - absorption matrix
        C_CORR - attenuation matrix
        lambdA - array of wavelengths
        deptH - array of depths (corresponding to spectra)"""
    with h5py.File(acs_fileNAME) as data: # Read in data as a .mat/hdf5 file
        A_CORR = data['A_CORR'][()].transpose() # Call in the absorption matrix
        C_CORR = data['C_CORR'][()].transpose() # Call in the attenuation matrix
        lamdA = data['a_wv'][()] # Call in the ac-s wavelengths (note: c is already interpolated to a wavelengths)
        deptH = data['deptH'][0] # Call in depth array
        return(A_CORR,C_CORR,lamdA,deptH)

def QAQC_acsDATA(A_CORR,C_CORR,lambdA,deptH):
    """QAQC_acsDATA performs QA/QC on ac-s data and removes spectra with at least one channel for which 
    a > c.
        Input variables:
        A_CORR - absorption matrix
        C_CORR - attenuation matrix
        lambdA - array of wavelengths
        deptH - array of depths (corresponding to spectra)
    Output variables:
        A_CORR, C_CORR, lambdA  - same variables as above, with contaminated spectra (a>c) removed"""
    ### 1. Create boolian indices and repository variables for QA/QC
    wvl_BOOL = lambdA <= 715 # Use bool logic to find all wavelengths less than 715 nm
    ac_IND = C_CORR > A_CORR # Use bool logic to find all instances where c > a (matrix)
    ac_QUIVER = [] # Create a list for acs rows for which a > c at some point in the row.
    ### 2. Index rows (spectra) for which a > c
    for b,aC in enumerate(ac_IND):
        # For loop cycles through the bool logic matrix row by row in order to determine if 
        # a > c at any point (bool = 0)
        neg_ARRAY = [i for i,j in enumerate(aC[wvl_BOOL[:,0]]) if j == 0] # Find channels where a > c
        if neg_ARRAY != []:
            # If at any point a > c in a given row
            ac_QUIVER.append(b)  # Add the index of the row to the "empty" list
    ac_QUIVER.sort(reverse=True) # Sort all a>c rows into reverse order
    ### 3. Remove contaiminated spectra (a > c) from absorption, attenuation, and depth
    for i in ac_QUIVER:
        # This for-loop detetes rows from a and c matrices for which a > c at ANY channel. 
        # It uses indices in descending order so as not to shift them before their deletions.
        A_CORR = np.delete(A_CORR, i, axis=0) # Delete row from a
        C_CORR = np.delete(C_CORR, i, axis=0) # Delete row from c
        deptH = np.delete(deptH, i, axis=0)   # Delete depth from d
    return(A_CORR,C_CORR,deptH) # Returns variables after contaminated rows removed

def QAQC_negative(A_CORR,C_CORR,lambdA,deptH):
    """QAQC_negative finds a/c values that are negative and removes their spectra from matrices
            Input variables:
                A_CORR - absorption matrix
                C_CORR - attenuation matrix
                lambdA - array of wavelengths
                deptH - array of depths (corresponding to spectra)
            Output variables:
                A_CORR, C_CORR, lambdA  - same variables as above, with contaminated spectra (a or c < 0) removed"""
    wvl_BOOL = lambdA <= 715 # Use bool logic to find all wavelengths less than 715 nm
    for i in range(2):
        # For-loop goes through two cycles. Cycle 1 (i = 0) deals with attenuation. Cycle 2 (i = 1)
        # deals with absorption
        if i == 0:
            ac_IND = C_CORR > 0 # Use bool logic to find all instances where c > a (matrix)
        else:
            ac_IND = A_CORR > 0 # Use bool logic to find all instances where c > a (matrix)
        ac_QUIVER = [] # Create a list for acs rows for which a > c at some point.
        for b,aC in enumerate(ac_IND):
            # For loop cycles through the bool logic matrix row by row in order to determine if 
            # a > c at any point (bool = 0)
            neg_ARRAY = [i for i,j in enumerate(aC[wvl_BOOL[:,0]]) if j == 0] # Find channels where a > c
            if neg_ARRAY != []:
                # If at any point a > c in a given row
                ac_QUIVER.append(b)  # Add the index of the row to the "empty" list
        ac_QUIVER.sort(reverse=True) # Sort all a>c rows into reverse order
        for i in ac_QUIVER:
            # This for-loop detetes rows from a and c matrices for which a > c at ANY channel. 
            # It uses indices in descending order so as not to shift them before their deletions.
            A_CORR = np.delete(A_CORR, i, axis=0) # Delete row from a
            C_CORR = np.delete(C_CORR, i, axis=0) # Delete row from c
            deptH = np.delete(deptH, i, axis=0) # Delete depth from d
    return(A_CORR,C_CORR,deptH)

def acs_BINNING(A_CORR,C_CORR,lambdA,deptH,**parameters):
    """acs_BINNING bins ac-s data in to depth bins as specified by the user. User must specify
    absorption
    Input variables:
        A_CORR - absorption matrix
        C_CORR - attenuation matrix
        lambdA - array of wavelengths
        deptH - array of depths (corresponding to spectra)
    Keyword arguments:
        biN_size - specifiy the depth bin size to use for the binning process
    Output variables:
        deptH_BINS - array of depth bins (median values)
        A_CORR_bin - depth-binned absorption spectra
        C_CORR_bin - depth-binned attenuation spectra"""
    biN_size = parameters['biN_size'] # Establish bin size from keyword arguments
    maX_deptH = np.ceil(max(deptH)) # Find the limit for binned depths
    deptH_BIN_EDGES = np.arange(0,maX_deptH+biN_size,biN_size) # Create an array of bin edges
    deptH_BINS = (deptH_BIN_EDGES[:-1]+deptH_BIN_EDGES[1:])/2  # Find median bin values
    A_CORR_bin = np.ones([len(deptH_BINS),len(lambdA)])*np.nan # Create nan matrix for binned a
    C_CORR_bin = np.ones([len(deptH_BINS),len(lambdA)])*np.nan # Create nan matrix for binned c
    for ii,d in enumerate(deptH_BINS):
        # For-loop indexes individual depths according to the proper bin and averages them according
        # to wavelength.
        difF_deptH = deptH - d # Find differences between depths and bin medians
        # Find indices of all depths that fall within the given bin 
        biN_IND = [i for i,j in enumerate(difF_deptH) if j > -(biN_size/2) and j <= (biN_size/2)] 
        naN_IND = [] # Create an empty array for potential non-existant bins (e.g. ac-s within a bin)
        if biN_IND != []:
            # If ac-s exist within a given depth bin
            A_CORR_bin[ii,:] = np.mean(A_CORR[biN_IND,:],axis=0) # Average selected a spectra
            C_CORR_bin[ii,:] = np.mean(C_CORR[biN_IND,:],axis=0) # Average selected c spectra
        else:
            # If no ac-s exist within a given depth bin
            naN_IND.append(ii) # Put index into the naN_IND list. Do not calculate means
    naN_IND.sort(reverse=True) # Reverse order of the empty bins
    for n in naN_IND:
        # Remove empty bins from the binned a & c matrices
        A_CORR_bin = np.delete(A_CORR_bin, n, axis=0) # Delete nans for binned a matrix
        C_CORR_bin = np.delete(C_CORR_bin, n, axis=0) # Delete nans for binned c matrix
        deptH_BINS = np.delete(deptH_BINS, n, axis=0) # Delete median depths corresponding to empty bins
    return(deptH_BINS,A_CORR_bin,C_CORR_bin)  

In [59]:
# 3. Define functions needed for pre-processing backscattering (HS6 data)

def bbp_ascii_LOAD(hs6_fileNAME):
    """bbp_ascii_LOAD reads a Hydrolight-compatible (binned) ascii hs6 file and places data
    into numpy variables taylored for easy access.
    Input variables:
        hs6_fileNAME - Hydrolight-compatible ascii file. This file needs to be sigma-corrected PRIOR to use.
    Output variables:
        bbp_deptH - array of depth bins (median values)
        bbp_data  - depth-binned backscattering coefficients (sigma-corrected prior to use)
        lambdA    - wavelength of HS6 channels placed into an array"""
    with open(hs6_fileNAME) as hs6_fid: # Open ascii file
        bbp_dict = {} # Create empty dictionary for ascii columns
        for i,ff in enumerate(hs6_fid): 
            # This for-loop reads in ascii data and organizes it into a dictionary
            if i == 9:
                # On the 10th line of the ascii header
                fielDS = ff # Create variable for the line
                fielDS = fielDS[:-1].split('\t') # Split the line by "tabs" and delete end of line character
            elif i == 10:
                # On the 11th line of the ascii header
                lambdA = ff # Create variable for the 11th line of the header (wavelengths)
                lambdA = lambdA.split('\t') # Split the wavelengths by "tabs"
                lambdA.pop(0) # Eliminate the first value, which is not a wavelength
                lambdA = np.asarray(lambdA,dtype=float) # Make wavelengths a numerical array
            elif i > 10:
                # On all lines below the wavelengths...
                ff = ff.split('\t') # Split data by tabs
                for g in fielDS:
                    # Index fielDS variable and apply the fields to the line of data. Index is used
                    # so assign bbp data to their correct fields (and dictionary key)
                    IND = fielDS.index(g) # Find the index of a field
                    if g in bbp_dict.keys():
                        # If the dictionary key has already been created
                        bbp_dict[g].append(ff[IND]) # Add data point to the dictionary key
                    else:
                        # If dictionary key is non-existant
                        bbp_dict[g] = [ff[IND]] # Create dictionary key and add data to it
        bbp_deptH = np.asarray(bbp_dict['Depth'],dtype=float) # Convert bbp depths to numpy array
        bbp_data = np.ones([len(bbp_deptH),len(lambdA)])*np.nan # Create a nan matrix for bbp data
        for i,l in enumerate(lambdA):
            # Column by column, for-loop fills in the nan matrix with dictionary data (see below)
            bbp_data[:,i] = np.asarray(bbp_dict['bb' + str(int(l))],dtype=float) 
            # Convert strings to numbers and add to dictionary (see above)
        return(bbp_deptH,bbp_data,lambdA)


def bbp_INTERPOLATE(bbp_deptH,bbp_data,bbp_lambdA,acs_lambdA,acs_deptH):
    """bbp_INTERPOLATE interpolates particulate for ac-s wavelenths. This function is a pre-requisite for 
    McKee et al.'s (2013) scattering correction approach.
    Input Variables:
        bbp_deptH - HS6 binned depths (medians)
        bbp_data - depth-binned particulate backscattering spectra
        bbp_lambdA - wavelengths of HS6 channels
        acs_lambdA - ac-s channel wavelengths
        acs_deptH - ac-s binned depths
    Output Variables
        bbp_acsINTERPOLATE - depth-binned particular backscattering spectra interpolated to ac-s wavelengths"""
    
    ### 1. Define functions within bbp_INTERPOLATE that will come in handy later on
    def coeff_CALC(x_1,y_1,x_2,y_2):
        """coeff_CALC calculates slopes and y-intercepts for two ordered pairs (or arrays of
        ordered pairs)
        Input Variables:
            x_1/x_2 - two wavelengths
            y_1/y_2 - two backscattering coefficients that correspons to wavelengths
        Output Variables:
            sloPE - slope of the line (AKA ordered pairs)
            y_int - y intercept of the line (AKA ordered pairs) or backscattering coefficient at theoretical wavelength of 0"""
        sloPE = (y_2 - y_1)/(x_2 - x_1) # Calculates a slope
        y_int = y_2 - (sloPE * x_2) # Calculates a y-intercept
        return(sloPE,y_int) 
    
    def y_CALC_linear(sloPE,y_int,x_val):
        """y_CALC_linear calculates a y value along a line corresponding to an "x-value".
        Input Variables:
            sloPE - slope of the line
            y_int - y-intercept of the line
            x_val - any value of x along the line. y_val will correspond with this value
        Output Variables:
            y_val - y-value that corresponds to the x-value on the line"""
        y_val = sloPE*x_val + y_int # Calculate y value using slope intercept form
        return(y_val)
    
    def acs_INDEXER(acs_d,acs_wvl,bbp_lambdA,bbp_deptH):
        """acs_INDEXER finds takes the depth and wavelength of a pair of ac-s absorption and attenuation values finds the index of the corresponding
        backscattering coefficient.
        Input Variables:
            acs_d - depth of the ac-s spectrum in question
            acs_wvl - wavelengths of the ac-s spectrum (channels)
            bbp_lambdA - wavelengths of the backscattering spectrum
            bbp_deptH - depth bins of the backscattering matrix
        Output Variables:
            wvl_IND - backscattering channel index for a pair of ac-s absorption/attenuation values
            deptH_IND - depth index for a pair of ac-s absorption/attenuation values"""
        deptH_diFF = np.abs(bbp_deptH - acs_d) # take absolute value of differences between ac-s and bbp depth bins
        deptH_IND  = np.where(deptH_diFF==min(deptH_diFF))[0][0] # Find depth index of the lowest absolute difference ac-s and backscattering depths 
        if acs_wvl < bbp_lambdA[0]:
            # If ac-s wavelength is outside (<) the spectral range of backscattering coefficients
            wvl_IND = 0 # Assign index of 0
        elif acs_wvl >= bbp_lambdA[-1]: 
            # If ac-s wavelength is outside (>) or equal to the highest backscattering wavelength
            wvl_IND = len(bbp_lambdA) - 2 # Assign the a specified index
        else:
            # If ac-s wavelenth is within the range of the backscattering spectrum (ideal scenario)
            for i in np.arange(len(bbp_lambdA)-1):
                # Search for the correct wavelength with the for-loop, which cycles through hs6 
                # wavelengths and determines whether the ac-s wavelength is inside.
                if bbp_lambdA[i] < acs_wvl <= bbp_lambdA[i+1]:
                    wvl_IND = i # Assign the wavelength index
                    break # Break the for-loop
        return(wvl_IND,deptH_IND)
    
    ### 2. Create matrices for slopes and y intercepts from which ac-s wavelengths will be interpolated
    bbp_yINT_Table  = np.ones([len(bbp_deptH), len(bbp_lambdA) - 1]) * np.nan # Create a nan matrix to put y intercepts
    bbp_sloPE_Table = np.ones([len(bbp_deptH), len(bbp_lambdA) - 1]) * np.nan # Create a nan matrix to put slopes
    for i,b in enumerate(bbp_data.transpose()):
        # For-loop fills bbp_yINT_Table and bbp_sloPE_Table with slopes and y intercepts. These values differ
        # depending on which hs6 wavelengths they fall between
        if i == len(bbp_lambdA)-1:
            # Indexing is over. Break the loop! 
            break
        else:
            # If indexing is NOT over (not the end of the line)
            y_2 = b # Second y ordered pair (bbp)
            y_1 = bbp_data[:,i+1] # First y ordered pair (bbp)
            x_2 = bbp_lambdA[i] # Second x ordered pair (lambda)
            x_1 = bbp_lambdA[i+1] # First x ordred pair (lambda)
            bbp_sloPE_Table[:,i],bbp_yINT_Table[:,i] = coeff_CALC(x_1,y_1,x_2,y_2) 
            # Calculate slopes and y intercepts for each hs6 wavelength
            
    ### 3. Interpolate bbp for ac-s wavelengths
    bbp_acsINTERPOLATE = np.ones([len(bbp_deptH),len(acs_lambdA)]) * np.nan # Create nan matrix for interpolated bbp
    for i,d in enumerate(bbp_deptH):
        for j,w in enumerate(acs_lambdA):
            # This nested for-loop will interpolate bbp for each ac-s depth and wavelength input
            # into the bbp_INTERPOLATE function.
            wvl_IND, deptH_IND = acs_INDEXER(d,w,bbp_lambdA,bbp_deptH) # Finds index for slope and y-intercept tables
            # Calculate interpolated bbp values (below)
            bbp_acsINTERPOLATE[i,j] = w * bbp_sloPE_Table[int(deptH_IND),int(wvl_IND)] + bbp_yINT_Table[int(deptH_IND),int(wvl_IND)] 
    return(bbp_acsINTERPOLATE)


In [60]:
# 4. Define the functions needed for calculating McKee-corrected absorption. This version of
# McKEE_SCATTER has the 

def McKEE_SCATTER(a_m,c_m,b_bp,rw):
    """McKEE_SCATTER goes through the iterative process described in McKee et al. (2013). (see flowchart). The function 
    is an iterative process that accepts a single triplet consisting of an absorption value, attenuation value, and a 
    backscattering coefficient. THESE VALUES SHOULD ALL CORRESPOND TO THE SAVE WAVELENGTH!
    Input variables:
        a_m - single absorption value (uncorrected for scatter)
        c_m - single attenuation value (uncorrected for scatter)
        b_bp - single particulate backscattering value (sigma-corrected) 
        rw - reflectance efficiency of ac-s absorption tube (coefficients listed in McKee)
    Output variables:
        an1 - McKee-corrected absorption value
        cn1 - McKee-corrected attenuation value
        b_p2 - McKee-corrected scatter value"""
    
    ### 1. Define functions within a the function   
    def fa_calc(x):
        """fa_calc will compute f_a variable based on empirical relationship between f_a and 
        backscattering fraction (x)
        Input Variables:
            x - backscattering fraction (backscattering coefficient/scatter)
        Output Variables:
            f_a - fraction of scattered light undetected by absorption meter"""
        ### 1. Set up empirical coefficients (as constants) for fa equation
        coeff0,coeff1,coeff2,coeff3,coeff4,coeff5,coeff6 = McKEE2013_COEFF(r_w=rw)
        # Calculate fa based on McKee et al. (2008)'s empirical formula
        f_a = coeff0 + coeff1*x + coeff2*(x**2) + coeff3*(x**3) + coeff4*(x**4) + coeff5*(x**5) + coeff6*(x**6)
        return(f_a)
    
    def fc_calc(x):
        """fc_calc will compute f_c variable based on empirical relationship between f_c and 
        backscattering fraction
        Input Variables:
            x - backscattering fraction (backscattering coefficient/scatter)
        Output Variables:
            f_c - fraction of scattered light undetected by attenuation meter"""
        # Set up empirical coefficients (as constants) for f_c equation. These remain constant no matter what rw
        coeff0 = 6.809e-3
        coeff1 = 8.502e-3
        coeff2 = -1.918e-2
        # Calculate f_c based on McKee et al. (2013)'s empirical formula
        f_c = coeff0/(coeff1 + x) + coeff2
        return(f_c)    

    def eCOEFFac(x):
        """eCOEFFac calculates error weighing functions for absorption and attenuation from 
        backscattering fraction (x).        
        Input Variables:
            x - backscattering fraction (backscattering coefficient/scatter)
        Output Variables:
            Ea - Calculate error weignt for absorption
            Ec - alculate error weight for attenuation"""
        fa = fa_calc(x) # Calculate fa1
        fc = fc_calc(x) # Calculate fc1
        Ea = fa/(1 - fc - fa) # Calculate error weignt for absorption
        Ec = fc/(1 - fc - fa) # Calculate error weight for attenuation
        return(Ea,Ec)
    
    ### 2. Before correcting for unmeasured ac-s scatter for iteration, determine starting variables
    keY = 0 # provides a failsafe for the upcoming while loop
    b_m = c_m - a_m # Calculate scatter from attenuation and absorption
    b_p0 = b_m*1.5 # Correct for particulate scatter
    b_frac0 = b_bp/b_p0 # Create initial backscattering fraction
    fa0 = fa_calc(b_frac0) # Calculate fa0
    fc0 = fc_calc(b_frac0) # Calculate fc0
    #return(b_m,b_p0,b_frac0,fa0,fc0)
    b_p1 = b_m/(1 - fa0 - fc0) # Calculate scatter #1 (b_p1)
    b_frac1 = b_bp/b_p1 # Create first backscattering fraction
    
    ### 3. Run the iteration as presribed by McKee et al. (2013)
    while 1:
        Ea1, Ec1 = eCOEFFac(b_frac1) # Calculates error weighing functions
        an1 = a_m - (Ea1 * b_m) # Correct absorption
        cn1 = c_m + (Ec1 * b_m) # Correct attenuation
        b_p2 = cn1 - an1 # Calculate scatter based on "Correct" absorption and attenuation
        b_frac2 = b_bp/b_p2 # Calculate "Corrected" backscattering fraction
        b_diFF = np.abs(b_frac2 - b_frac1) # Calculate difference between "Corrected" and first iteration fraction
        if b_diFF < 0.001 or keY > 9:
            if keY == 9:
                print('Key')
            # If the scattering coefficient was appropriately permutated, or if there were already
            # 11 permutations
            break # break the while loop!
        else:
            # If the scattering permutation is insufficient
            b_frac1 = b_frac2 # Convert the second backscattering fraction to the first
            keY+=1 # Record the iteration by increaing key
    return(an1,cn1,b_p2) # Return calculated variables


In [61]:
# 5. Define functions necessary for formatting scatter-corrected ac-s data.

def createFolder(directory):
    """ createFolder searches for a dirctory specified by the user. If there is none, it creates one"""
    try:
        if not os.path.exists(directory): # If the folder doesn't exist
            os.makedirs(directory) # Create a folder
    except OSError: # If there is an error other than a non-existant folder
        print ('Error: Creating directory. ' +  directory) # report error and shut down createFolder

def Boostrap_FileWriter(BOOTSRAP_MATRIX,**parameters):
    """Creates a Hydrolight-compatible ascii file using depth-binned IOP data. File is output in a separate folder.
    This folder is placed in the same directory as the original file containing non-binned IOP data.
    Inputs:
        BOOTSRAP_MATRIX - Depth-binned IOP matrix
    keyword arguments (in order of appearance):
        acs_file - the mat/hdf5 file from which binned data originated
        TREAT - the sigma correction used on bbp data
        hs6_file - the file containing binned bbp data
        station - station number/name
        BIN - depth bin size
        lambdA - array of ac-s wavelengths
        DIR - specify the directory to place the newly-created McKee-corrected ac-s file 
    Outputs:
        Hyrolight-compatible file of binned ac-s data. This output is NOT a variable."""
    ### 1. Create a dictionary containing all important information for file header and column titles. This information
    ### is found in key-word arguments.
    abs_str = '\ta' # Absorption marker for column headers
    atn_str = '\tc' # Absorption marker for column headers
    File_acs_new = parameters['acs_file'][:-7]  + parameters['TREAT'] + '.txt' # Create name for new ac-s file (final product)
    rW = str(parameters['r_w']) # Take r_w (reflectance efficiency) and convert into a string
    headER = {}
    headER['0'] = 'Total absorption (a) and attenuation (c) measurements\n' # Header line 1 (Data type)
    headER['1'] = 'Corrected: ' + dt.now().strftime("%d/%m/%Y %H:%M:%S") + '\n' # Header line 2 (Processing date)
    headER['2'] = 'Instrument: ac-s\n' # Header line 3 (Instrument type)
    headER['3'] = 'ac-s file name: ' + parameters['acs_file'] + '\n' # Header Line 4 (Name of the original ac-s file)
    headER['4'] = 'hs6 file name: ' + parameters['hs6_file'] + '\n' # Header Line 5 (Name of the backscattering coefficients file)
    headER['5'] = 'Processed using McKee et al. (2013), r_w = ' + rW + '\n' # Header Line 6 (Name of the scatter correction method)
    headER['6'] = 'Station: ' + str(parameters['station']) + '\n' # Header Line 7 (sampling station name)
    headER['7'] = 'Bin Size = ' + str(parameters['BIN']) + ' m\n' # Header Line 8 (depth bin size)
    headER['8'] = 'Data have been processed using code written and made available by Jesse Bausell (email: jbausell@ucsc.edu, GitHub: JesseBausell).\n' # Header Line 9 (shameless self promotion)
    headER['9'] = 'Depth' # Header line 10. Start of column titles (more to be added later)
    headER['10'] = str(len(parameters['lambdA'])) # Header Line 11. Actual number of ac-s channels (more to be added later) 
    
    ### 2. Develop lines 10 and 11 of the file header.
    fieldS_c = ''# Create an empty string for column headers/titles     
    for l in parameters['lambdA']: 
        # For-loop cycles through ac-s channels (wavelengths) and creates column headers for absorption and 
        # attenuation (Header Line 10). It also places wavelengths themselves on line 11 per Hydrolight formatting 
        # instructions
        headER['9'] += abs_str + str(l[0]) # Add 'tab' + 'a' + wavelength to the Header Line 10
        fieldS_c += atn_str + str(l[0]) # Add 'tab' + 'c' + wavelength to the segment to be added to Header Line 10
        headER['10'] += '\t' + str(l[0]) # Add 'tab' + wavelength to segment to be added to Header Line 11
    headER['9'] += fieldS_c + '\n' # Add attenuation column headers and end of line character to Header Line 10
    headER['10'] += '\n' # Add end of line character to Header Line 10
    
    ### 3. Take absorption and attenuation data and create a Hydrolight-compatible ac-s file
    with open(parameters['DIR']+File_acs_new, 'w', newline='') as txtfile: # Create a new ascii file
        for i in np.arange(len(headER)):
            # for-loop writes the header metadata into the new file
            txtfile.write(headER[str(i)]) # Write header line into file
        for nn in np.arange(len(BOOTSRAP_MATRIX[:,0])):
            # for-loop writes the binned IOP data into the new file
            if ~np.isnan(BOOTSRAP_MATRIX[nn,1]):
                # if the last line is not reached
                datUM = str(BOOTSRAP_MATRIX[nn,:].tolist())[1:-1] # Convert row of data from numpy array into comma-separated string. Remove brackets
                datUM = datUM.replace(', ','\t') # replace commas with tabs
                txtfile.write(datUM + '\n') # Write the data into the file
                keY = nn # this variable preserves last row of actual values e.g. (not nan values)
        BOOTSRAP_MATRIX[keY,0] = -1 # After data has been written into the file, replace the first column in the last line (depth) with -1
        datUM = str(BOOTSRAP_MATRIX[nn,:].tolist())[1:-1] # Re-convert last row of data to comma-separated string
        datUM = datUM.replace(', ','\t') # remove end of line characters
        txtfile.write(datUM + '\n') # re-write the last line of code into the file

In [62]:
acs_dir

'/Users/JBausell/Desktop/COAST_18_acsPRE.mat'

In [63]:
# 6. Now that all of the libraries are imported and the functions are defined,  

### 1. Create starting variables to work with

# Create dictionary of file appendages (to be ID which rW value was used) along with different rW values described in 
# McKee et al. (2013)
TREAT = {'_Mc13_1000':1,'_Mc13_0999':0.999,'_Mc13_0998':0.998,'_Mc13_0995':0.995,
         '_Mc13_0990':0.99,'_Mc13_0985':0.985,'_Mc13_0980':0.98,'_Mc13_0970':0.97,
         '_Mc13_0960':0.96,'_Mc13_0950':0.95}
depTHBIN = 0.5 # User specifies depth-bin. 
station_name = 'CST18' # User specifies station

# Select ac-s file to process. Create variables that will be used down the road
acs_fileNAME = fd.askopenfilename()    # Select ac-s file to apply McKee et al. (2013) scatter correction
acs_ind = acs_fileNAME.rfind('/')          # Find the end of the directory and the beginning of the file name
acs_dir = acs_fileNAME[:acs_ind+1]      # Separate directory from file pathway
acs_fileNAME = acs_fileNAME[acs_ind+1:] # Separate file name from file pathway
acs_ind2 = acs_file.rfind('.')         # Find the beginning of the filename extension
acs_fileBASE = acs_fileNAME[:acs_ind2] # Create file base. This will be used to develop and create new files
McKee_DIR = acs_dir + '/McKee_2013/'   # Create new directory as repository for newly created ac-s files (scatter corrected)
createFolder(McKee_DIR)                # Create directory for aforementioned repository (see above line)
# Select binned (Hydrolight-compatible) backscattering coefficient file. If using HS6, this file should ALREADY be sigma-corrected
bbp_fileNAME = fd.askopenfilename()    # Select bbp file to use in support of McKee et al.'s (2013) scatter correction approach

### 2. Load, process, and bin uncorrected ac-s data
A_CORR,C_CORR,acs_lambdA,acs_deptH = acs_matlab_LOAD(acs_dir+acs_fileNAME) # Load ac-s .mat/hdf5 file
A_CORR,C_CORR,deptH = QAQC_acsDATA(A_CORR,C_CORR,acs_lambdA,acs_deptH)     # Flag and remove problematic ac-s spectra (a>c)
A_CORR,C_CORR,deptH = QAQC_negative(A_CORR,C_CORR,acs_lambdA,deptH)        # Flag and remove problematic ac-s spectra (a or c<0)
deptH_BINS_acs,A_CORR_bin,C_CORR_bin = acs_BINNING(A_CORR,C_CORR,acs_lambdA,deptH,biN_size=depTHBIN) # Bin data into user-specified depth-bin
nanIND = [i for i,a in enumerate(A_CORR_bin[:,0]) if not np.isnan(a)]      # locate and flag NaNs in binned ac-s data
A_CORR_bin = A_CORR_bin[nanIND,:] # Remove NaNs from binned absorption
C_CORR_bin = C_CORR_bin[nanIND,:] # Remove NaNs from binned attenuation
deptH_BINS_acs = deptH_BINS_acs[nanIND] # Remove NaNs from depth bins

### 3. Load and interpolate backscattering data to be consistent with use for 
bbp_deptH,bbp_data,bbp_lambdA = bbp_ascii_LOAD(bbp_fileNAME) # Read in the HE53-formatted ascii file
bbp_acsINTERPOLATE = bbp_INTERPOLATE(bbp_deptH,bbp_data,bbp_lambdA,acs_lambdA,deptH_BINS_acs) 
bbp_ind = bbp_fileNAME.rfind('/') # Find the end of the directory and the beginning of the file name
bbp_fileNAME = bbp_fileNAME[bbp_ind+1:] # Separate the file name from the directory

### 4. Apply McKee et al. (2013) Scatter Correction Approach to binned ac-s data
for t in TREAT.keys():
# for-loop goes through each of the 10 rw's, as laid out in McKee et al. (2013). For each rw, the loop applies 
# McKee et al.'s (2013) approach, and then formats scatter-corrected ac-s data into a Hydrolight-compatible ascii file

#     hs6_diR = acs_diR + t + '/' # create folder pathway for McKee et al. (2013) SCA
#     createFolder(hs6_diR) # create folder for McKee et al. (2013 SCA) if not in existence
#     copy_tree(acs_diR + '_Mc08/', hs6_diR) # Copy all contents from McKee_tools into new folder
#     hs6_fileNAME = STN[s] + '_bbp_bin0.5_Mc08.txt'  # Select directory containing m-files        
#     bbp_deptH,bbp_data,bbp_lambdA = bbp_ascii_LOAD(bbp_fileNAME) # Read in the HE53-formatted ascii file
#     ### Interpolate bbp for ac-s wavelengths
#     bbp_acsINTERPOLATE = bbp_INTERPOLATE(bbp_deptH,bbp_data,bbp_lambdA,acs_lambdA,deptH_BINS_acs)

    ## 4a. Create empty matrices to act as repositories for finished products and run nested for-loop!
    A_CORR_McKee = np.ones(A_CORR_bin.shape)*np.nan # empty nan matrix for McKee absorption
    C_CORR_McKee = np.ones(C_CORR_bin.shape)*np.nan # empty nan matrix for McKee attenuation
    B_CORR_McKee = np.ones(C_CORR_bin.shape)*np.nan # empty nan matrix for McKee scatter
    
    for acs_IND,d_acs in enumerate(deptH_BINS_acs):
        # for-loop cycles through ac-s spectra, selecting them one at a time for McKee correction. Because the for-loop
        # is nested, an additional for-loop takes absorption, attenuation, and backscattering coefficients from each
        # spectrum one-by-one, and applies McKee et al. (2013) to them.
        ## 4b. Select the appropriate backscattering spectrum (same or similar depth) for selected depth-binned ac-s spectra
        difF_acs = np.abs(bbp_deptH - d_acs) # Find absolute differences between backscattering depth bins and ac-s spectra
        bbp_IND = np.where(difF_acs==min(difF_acs))[0][0] # Find index of the smallest absolute difference (AKA closest binned depth)
        a_array = A_CORR_bin[acs_IND,:] # Select mean absorption spectrum at the appropriate depth
        c_array = C_CORR_bin[acs_IND,:] # Select mean attenuation spectrum at the appropriate depth
        bbp_array = bbp_acsINTERPOLATE[bbp_IND,:] # Select interpolated bbp spectrum based on the appropriate depth
        for i,a_m in enumerate(a_array):
            ### 4c. This for-loop goes through the selected absorption, attenuation, and interpolated 
            # backscattering spectra channel by channel. It takes each uncorrected absorption, and its 
            # corresponding attenuation and backscattering coefficient and calculates McKee et al.
            # (2013) scatter channel by channel.
            c_m = c_array[i] # Find uncorrected attenuation based on absorption index
            b_bp = bbp_array[i] # Find backscattering coefficient based on absorption index
            an1,cn1,bn1 = McKEE_SCATTER(a_m,c_m,b_bp,TREAT[t]) # Correct for absorption, attenuation, and scatter
            A_CORR_McKee[acs_IND,i] = an1 # Place absorption in the correct matrix
            C_CORR_McKee[acs_IND,i] = cn1 # Place attenuation in the correct matrix
            B_CORR_McKee[acs_IND,i] = bn1 # Place scatter in the correct matrix
    IOP_MATRIX = np.round(np.append(A_CORR_McKee,C_CORR_McKee,axis=1),decimals=6) # Round answers to three decimal places
    IOP_MATRIX = np.append(np.reshape(deptH_BINS_acs,[len(deptH_BINS_acs),1]),IOP_MATRIX,axis=1) # Attach binned depths to lefthand side
    Boostrap_FileWriter(IOP_MATRIX,DIR=McKee_DIR,acs_file=acs_fileNAME,hs6_file=bbp_fileNAME,station=station_name,lambdA=acs_lambdA,BIN=depTHBIN,r_w=TREAT[t],TREAT =t)
#        batch_FILE = 'I' + s + '_bin_05_PCT100' + t + '.txt'
#     batch_FILE = 'I' + s + '_bin_05_PCT100' + '.txt' # Generic batch file name. This will be used to create proper batch file
#     HE53_batch_WRITER(batch_diR+batch_FILE,STN=s,Station=STN[s],TRMT=t,TREATMENT='_bbp_bin0.5_Mc08.txt') # 